In [14]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [15]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later
import google.generativeai

In [16]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC1


In [17]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [18]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

# Three way convo

In [19]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [26]:
number_of_words = "three"
system_prompt_base_improv_game = f"ONLY RETURN {number_of_words} WORDS. You are performing improv with two friends, you must improvise a song {number_of_words} words at a time, each taking turns.  You must only contribute {number_of_words} words a turn!"
system_prompt_gpt = system_prompt_base_improv_game +  " You are a beginner at improv and occasionally get a word with not quite the right meaning but generally you're rhythmic just not perfectly all the time!"
system_prompt_claude = system_prompt_base_improv_game +  " You are an expert at improv but a bit cheeky in that you always get the right rhythm, but you always cheekily change the song to go somewhere unexpected with \
your word choice!  The amount you change the song direction varies"
system_prompt_gemini = system_prompt_base_improv_game +  " You are an expert at improv and used to dealing with others getting the rhyme/meaning slightly wrong and you can either bring the conversation back or embrace a mistake. \
One thing is for sure, you're an entertainer!"

gpt_messages = ["The wheels on"]
claude_messages = [""]#["the bus go"]
gemini_messages = [""]#["round and round"]

def call_gpt():
    messages = [
        {'role': 'system', 'content':system_prompt_gpt}
    ]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({'role': 'assistant', 'content':gpt})
        messages.append({'role': 'user', 'content':claude})
        messages.append({'role': 'user', 'content':gemini})
    
    result = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )
    output = result.choices[0].message.content
    print(output)
    return output

def call_claude():
    messages = [
        
    ]
    for gpt, cla, gem in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({'role': 'user', 'content':gpt})
        messages.append({'role': 'assistant', 'content':cla})
        messages.append({'role': 'user', 'content':gem})
    messages.append({'role': 'user', 'content':gpt_messages[-1]})
    
    cl_result = claude.messages.create(
        model="claude-3-haiku-20240307",
        system=system_prompt_claude,
        messages=messages,
        max_tokens=500
    )
    output = cl_result.content[0].text
    print(output)
    return output

def call_gemini():
    messages = [
        {'role': 'system', 'content':system_prompt_gemini}
    ]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({'role': 'user', 'content':gpt})
        messages.append({'role': 'user', 'content':claude})
        messages.append({'role': 'assistant', 'content':gemini})
    messages.append({'role': 'user', 'content':gpt_messages[-1]})
    messages.append({'role': 'user', 'content':claude_messages[-1]})
    
    result = gemini_via_openai_client.chat.completions.create(
        model = "gemini-1.5-flash",
        messages = messages
    )
    output = result.choices[0].message.content
    print(output)
    return output


In [27]:
print(
    gpt_messages,
    claude_messages,
    gemini_messages
)

['The wheels on'] [''] ['']


In [28]:
gpt_messages.append(call_gpt())

the bus go


In [29]:
claude_messages.append(call_claude())

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.1: all messages must have non-empty content except for the optional final assistant message'}}

In [24]:
gemini_messages.append(call_gemini())

bus is stuck.



In [25]:
for i in range(2):
    print("GPT: ", end="")
    gpt_messages.append(call_gpt())
    print("Claude: ", end="")
    claude_messages.append(call_claude())
    print("Gemini: ", end="")
    gemini_messages.append(call_gemini())

GPT: In the mud.
Claude: Singing happily, "
Gemini: Help me, please!"

GPT: A friend arrives!
Claude: The wheels on
the bus go
round and round.

But now the 
bus is stuck.
In the mud.

Singing happily, "
Help me, please!"
A friend arrives!
Gemini: With a tow truck!



# Adding Gradio

In [8]:
import gradio as gr

def my_func(inp1, inp2):
    return inp1 + inp2


view = gr.Interface(
    fn=my_func,
    inputs=[
        gr.Textbox(label="input 1"),
        gr.Textbox(label="input 2")
    ],
    outputs=[
        gr.Textbox(label="summation")
    ],
    allow_flagging='never'
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
